Parse TLE to KeplerInputs

In [37]:
import sys
sys.path.append('/workspaces/ma-max-wendler/scripts/keplertraces')  # Add the directory to the search path

In [38]:
import tleparse

In [39]:
# inputs = tleparse($tlespath$), i.e.
path = "/workspaces/ma-max-wendler/scripts/keplertraces/tles/examples_requested_at_2023-10-23-11-06-08/iridium-NEXT_2023-10-22-21-39-19.txt"
tles = tleparse.read(path)

In [40]:
# objects of the TLE-tool that among other things parses TLE values
tle = tles[0]
tle

TLE(name='IRIDIUM 106', norad='41917', classification='U', int_desig='17003A', epoch_year=2023, epoch_day=295.91774854, dn_o2=1.91e-06, ddn_o6=0.0, bstar=6.0989e-05, set_num=999, inc=86.3949, raan=133.2138, ecc=0.0001982, argp=93.4819, M=266.6603, n=14.34216513, rev_num=35442)

For comparison: IRIDIUM 106 TLE

```
IRIDIUM 106
1 41917U 17003A   23295.91774854  .00000191  00000+0  60989-4 0  9995
2 41917  86.3949 133.2138 0001982  93.4819 266.6603 14.34216513354425
```

Convert to KeplerInputs instance, contains two conversions:

In [41]:
from poliastro.twobody.angles import M_to_E, E_to_nu
import astropy.units as u
from math import pi

In [42]:
# mean anomaly to true anomaly via poliastro functions
ecc = u.Quantity(tle.ecc, unit=u.one)
mean_anom = u.Quantity(tle.M, unit=u.deg)
ecc, mean_anom

(<Quantity 0.0001982>, <Quantity 266.6603 deg>)

In [43]:
mean_anom_rad = mean_anom.to(u.rad)
# mean_anom_rad_negpos = u.Quantity(pi,unit=u.rad) - mean_anom.to(u.rad)
# -pi (when pi passed) + (mean_anom_rad - pi) (overlap over pi)
mean_anom_rad_negpos = mean_anom.to(u.rad) - u.Quantity(2*pi,unit=u.rad) 

mean_anom_rad, mean_anom_rad_negpos, 2*pi + mean_anom_rad_negpos.value

(<Quantity 4.65410022 rad>, <Quantity -1.62908509 rad>, 4.654100219355835)

In [44]:
import tletools

In [45]:
"""
**Warning**

    The mean anomaly must be between -π and π radians.
    The eccentricity must be less than 1.
"""
    
# tle-tools-based true anomaly
true_anom_t = u.Quantity(tletools.utils.M_to_nu(mean_anom_rad_negpos, tle.ecc), unit=u.rad).to(u.deg)
true_anom_t, true_anom_t.to(u.rad)

(<Quantity -93.36237315 deg>, <Quantity -1.62948081 rad>)

In [46]:
# poliastro based True anomaly -> can use mean anomaly in degrees
true_anom_p = E_to_nu( M_to_E( mean_anom_rad_negpos, ecc ), ecc )
_true_anom_p = E_to_nu( M_to_E( mean_anom_rad, ecc ), ecc )
true_anom_p, _true_anom_p, true_anom_p.to(u.deg)

(<Quantity -1.62948081 rad>,
 <Quantity -1.62948081 rad>,
 <Quantity -93.36237315 deg>)

In [47]:
from orbital.utilities import true_anomaly_from_mean

In [48]:
# OrbitalPy based True anomaly
# uses radian in 0,2pi range (Mnorm = fmod(M, 2 * pi))
true_anom_o = true_anomaly_from_mean(ecc.value, mean_anom_rad.value)
true_anom_o, u.Quantity( -1 * (2*pi - true_anom_o), unit=u.rad).to(u.deg)

(4.653704498287206, <Quantity -93.36237315 deg>)

In [49]:
import rebound

In [50]:
# rebound-based True anomaly
# can't really tell in which range M should be, as its C code
true_anom_r = rebound.M_to_f(tle.ecc, mean_anom_rad_negpos.value)
true_anom_r, u.Quantity(-360, unit=u.deg) + u.Quantity(true_anom_r, unit=u.rad).to(u.deg)

(4.653704498287205, <Quantity -93.36237315 deg>)

In [51]:
from skyfield import keplerlib

In [52]:
# can take mean anomalies in both ranges
ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad.value)
_ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad_negpos.value)
true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, ecc_anom)
_true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, _ecc_anom)

true_anom_s, u.Quantity(true_anom_s, unit=u.rad).to(u.deg), _true_anom_s

(-1.629480808892381, <Quantity -93.36237315 deg>, -1.629480808892381)

Semi-major axis

In [53]:
# tletools 
tle.a

7155.808268901222

In [54]:
from astropy.constants import GM_earth

In [55]:
# researched formula
secs_per_revolution = 86400 / tle.n
semimajoraxis = u.Quantity((secs_per_revolution**2 * GM_earth.value / (4 * pi**2) )**(1/3) / 1000, unit=u.km)
semimajoraxis

<Quantity 7155.80801877 km>

TEME coordinates from poliastro orbit

In [56]:
# creates kepler orbit
orb = tle.to_orbit()
orb

7154 x 7157 km x 86.4 deg (GCRS) orbit around Earth (♁) at epoch 2023-10-22T22:01:33.473856000 (UTC)

Show that equivalent to reference calculations from literature / validated above:

In [57]:
from keplerinputs import KeplerInputs
from poliastro.twobody import Orbit
from poliastro.bodies import Earth

In [58]:
kepler_inputs = KeplerInputs(tle.name, tle.ecc, tle.inc, tle.raan, tle.argp, mean_anom=tle.M, mean_motion=tle.n, epoch=tle.epoch)
orb = Orbit.from_classical(
            Earth, 
            a=kepler_inputs.semimajoraxis, 
            ecc=kepler_inputs.ecc, 
            inc=kepler_inputs.inc, 
            raan=kepler_inputs.raan, 
            argp=kepler_inputs.argp, 
            nu=kepler_inputs.true_anom, 
            epoch=kepler_inputs.epoch)
orb

7154 x 7157 km x 86.4 deg (GCRS) orbit around Earth (♁) at epoch 2023-10-22T22:01:33.473856000 (UTC)

=> tletools orbit creation is equivalent.

Now: compare ephem coordinates to TEME coordinates from C++ SGP4

In [59]:
from datetime import datetime
from astropy.time import Time

In [60]:
# following avg. epoch of all sats, which is encoded in name of TLEs file, 'iridium-NEXT_2023-10-22-21-39-19'
start_datetime = datetime.strptime("2023-10-22-21-39-19", '%Y-%m-%d-%H-%M-%S')
sim_start_time = Time(start_datetime, format='datetime', scale='utc')
sim_start_time

<Time object: scale='utc' format='datetime' value=2023-10-22 21:39:19>

In [61]:
from poliastro.twobody.sampling import EpochsArray
from poliastro.util import time_range

In [63]:
# could use different propagator for EpochsArray, which currently is 'FarnocchiaPropagator'
update_interval = (1 << u.s)
periods = 25
# first coordinate change at first sim second
ephem = orb.to_ephem(strategy=EpochsArray(epochs=time_range(sim_start_time + update_interval, spacing=update_interval, periods=periods)))
ephem

Ephemerides at 25 epochs from 2023-10-22 21:39:20 (UTC) to 2023-10-22 21:39:44 (UTC)

In [66]:
# ICRS coordinates, but orbit above specifies GCRS?
# => GCRS is specific version of ICRS
"""
Regardless of how the Orbit is created, 
the implicit reference system is an inertial one. 
For the specific case of the Solar System, this can be assumed 
to be the International Celestial Reference System or ICRS.
"""

cartesian_trace = ephem.sample(ephem.epochs)
# example coordinates
cartesian_trace[:3]

<CartesianRepresentation (x, y, z) in km
    [(-566.52604985, 1249.35180412, -7024.52179864),
     (-571.61236662, 1254.64088416, -7023.16674788),
     (-576.69806194, 1259.92860018, -7021.80406161)]
 (has differentials w.r.t.: 's')>

In [79]:
from astropy.coordinates import GCRS, WGS84GeodeticRepresentation, SkyCoord

In [78]:
wgs84_trace = WGS84GeodeticRepresentation.from_cartesian(cartesian_trace)
wgs84_trace

<WGS84GeodeticRepresentation (lon, lat, height) in (rad, rad, m)
    [(1.99652053, -1.3790561 , 799681.14816387),
     (1.99829611, -1.37807636, 799673.05393379),
     (2.00005396, -1.37709602, 799664.91509588),
     (2.00179432, -1.37611511, 799656.73168453),
     (2.00351745, -1.37513362, 799648.50373428),
     (2.00522361, -1.37415156, 799640.2312799 ),
     (2.00691303, -1.37316895, 799631.9143563 ),
     (2.00858596, -1.37218579, 799623.5529986 ),
     (2.01024262, -1.37120209, 799615.14724212),
     (2.01188325, -1.37021785, 799606.69712235),
     (2.01350808, -1.36923309, 799598.20267495),
     (2.01511733, -1.36824782, 799589.6639358 ),
     (2.01671121, -1.36726203, 799581.08094094),
     (2.01828994, -1.36627574, 799572.45372662),
     (2.01985373, -1.36528896, 799563.78232924),
     (2.02140279, -1.36430168, 799555.06678541),
     (2.02293732, -1.36331393, 799546.30713193),
     (2.02445752, -1.3623257 , 799537.50340577),
     (2.02596359, -1.361337  , 799528.65564409),
    

In [81]:
skycoord_gcrs = SkyCoord(cartesian_trace, frame='gcrs', obstime=ephem.epochs)
skycoord_gcrs

<SkyCoord (GCRS: obstime=[datetime.datetime(2023, 10, 22, 21, 39, 20)
 datetime.datetime(2023, 10, 22, 21, 39, 21)
 datetime.datetime(2023, 10, 22, 21, 39, 22)
 datetime.datetime(2023, 10, 22, 21, 39, 23)
 datetime.datetime(2023, 10, 22, 21, 39, 24)
 datetime.datetime(2023, 10, 22, 21, 39, 25)
 datetime.datetime(2023, 10, 22, 21, 39, 26)
 datetime.datetime(2023, 10, 22, 21, 39, 27)
 datetime.datetime(2023, 10, 22, 21, 39, 28)
 datetime.datetime(2023, 10, 22, 21, 39, 29)
 datetime.datetime(2023, 10, 22, 21, 39, 30)
 datetime.datetime(2023, 10, 22, 21, 39, 31)
 datetime.datetime(2023, 10, 22, 21, 39, 32)
 datetime.datetime(2023, 10, 22, 21, 39, 33)
 datetime.datetime(2023, 10, 22, 21, 39, 34)
 datetime.datetime(2023, 10, 22, 21, 39, 35)
 datetime.datetime(2023, 10, 22, 21, 39, 36)
 datetime.datetime(2023, 10, 22, 21, 39, 37)
 datetime.datetime(2023, 10, 22, 21, 39, 38)
 datetime.datetime(2023, 10, 22, 21, 39, 39)
 datetime.datetime(2023, 10, 22, 21, 39, 40)
 datetime.datetime(2023, 10, 2

In [75]:
gcrs_trace = GCRS(cartesian_trace, obstime=ephem.epochs)
gcrs_trace[:3]

<GCRS Coordinate (obstime=[datetime.datetime(2023, 10, 22, 21, 39, 20)
 datetime.datetime(2023, 10, 22, 21, 39, 21)
 datetime.datetime(2023, 10, 22, 21, 39, 22)], obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, km)
    [(114.39220013, -78.94994311, 7157.21581309),
     (114.49393343, -78.8934966 , 7157.2156328 ),
     (114.59465043, -78.83701628, 7157.21545099)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [(2.22638824e+12, 6.41079902e+12, -0.00017952),
     (2.21521419e+12, 6.41466908e+12, -0.00018105),
     (2.20414729e+12, 6.41848056e+12, -0.00018258)]>